In [1]:
# Import standard libraries
import os, datetime
from pathlib import Path
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle as pl
import igraph
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import bokeh
import hvplot.pandas
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', 'matplotlib')

import bokeh.palettes
from bokeh.plotting import figure, show, output_notebook
output_notebook()
import seaborn as sb
from scipy.sparse import csr_matrix
from scipy.sparse import save_npz, load_npz
# import cairo
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import zscore
from scipy.sparse.linalg import norm
import networkx as nx
from pyvis.network import Network

## Load environment variables from .env file and find project root
import sys
from dotenv import load_dotenv, find_dotenv
load_dotenv()
PROJECT_ROOT = Path(find_dotenv()).parent
# data_path = Path(PROJECT_ROOT, 'results', 'eyemap')
store_path = Path(PROJECT_ROOT, 'results','AOTU_Connectivity')
sys.path.append(str(PROJECT_ROOT.joinpath('src')))


from utils import olc_client
c = olc_client.connect(verbose=True)

from utils.celltype_conn_by_roi import CelltypeConnByRoi
from utils.celltype_conn_plotter import CelltypeConnPlotter
from utils.plotter import plot_cns, save_figure, get_skeletons, get_skeleton, get_meshes, get_mesh, show_figure
from utils.helper import slugify
from utils.neuron_bag import NeuronBag

# Import neuPrint specific libraries
from neuprint import Client, fetch_neurons, NeuronCriteria as NC, fetch_neurons, fetch_simple_connections, fetch_adjacencies, connection_table_to_matrix, merge_neuron_properties, NotNull, fetch_synapse_connections, fetch_neurons,fetch_primary_rois, fetch_all_rois, fetch_synapses, fetch_roi_hierarchy
from neuprint.utils import connection_table_to_matrix

Loading BokehJS ...

Connected to https://neuprint-cns.janelia.org[cns].
Client: neuprint-python v1.7.4
User: aishahamid201@gmail.com [readwrite]



In [2]:
PROJECT_ROOT

PosixPath('/Users/hamida/Documents/GitHub/male-drosophila-visual-system-connectome-code')

In [3]:
# Specify LC10 Neurons of Interest by instance or type (save neuron properties & synapse ROIs if you wish):

## For all LC10 neurons on the right side:
#LC10_R_criteria = NC(instance = ('LC10a_R', 'LC10b_R','LC10c-1_R','LC10c-2_R','LC10d_R', 'LC10e_R', 'LC10unclear_R'))
#LC10s_neuron_df, roi_counts_df = featch(LC10_R_criteria)

## For only LC10a neurons on right side:
LC10a_R_criteria =  NC(instance = ('LC10a_R'));
LC10a_neuron_df,roi_counts_df = fetch_neurons(LC10a_R_criteria)

## To save neuron properties and synapse ROIs as .csv files:
LC10a_neuron_df.to_csv(PROJECT_ROOT/'AOTU_results/LC10a_R_NeuronProperties.csv')#
roi_counts_df.to_csv(PROJECT_ROOT/'AOTU_results/LC10a_R_SynapsePerROICounts.csv')

## to list LC10s, uncomment:
LC10a_neuron_df

/Users/hamida/Documents/GitHub/male-drosophila-visual-system-connectome-code/.pixi/envs/default/lib/python3.12/site-packages/neuprint/client.py:609: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neuron_props_val = df_results.iloc[0][0]
/Users/hamida/Documents/GitHub/male-drosophila-visual-system-connectome-code/.pixi/envs/default/lib/python3.12/site-packages/neuprint/client.py:619: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neuron_props_val = df_results.iloc[0][0]


,bodyId,instance,type,pre,post,downstream,upstream,size,status,statusLabel,...,fruDsx,trumanHl,matchingNotes,celltypePredictedNt,anteriorCervicalRootPosition,celltypePredictedNtConfidence,assignedOlHex1,supertype,inputRois,outputRois
0,21065,LC10a_R,LC10a,392,1857,2694,1857,624799190,Traced,Roughly traced,...,fru_high,None,None,acetylcholine,None,0.576722,None,None,"[AOTU(R), CentralBrain, CentralBrain-unspecifi...","[AOTU(R), CentralBrain, CentralBrain-unspecifi..."
1,21199,LC10a_R,LC10a,398,1649,2786,1649,616008264,Traced,Roughly traced,...,fru_high,None,None,acetylcholine,None,0.576722,None,None,"[AOTU(R), CentralBrain, CentralBrain-unspecifi...","[AOTU(R), CentralBrain, LO(R), LO_R_col_20_14,..."
2,21238,LC10a_R,LC10a,375,1886,2658,1886,616583509,Traced,Roughly traced,...,fru_high,None,None,acetylcholine,None,0.576722,None,None,"[AOTU(R), CentralBrain, LO(R), LO_R_col_19_17,...","[AOTU(R), CentralBrain, CentralBrain-unspecifi..."
3,21426,LC10a_R,LC10a,356,1548,2445,1548,607347200,Traced,Roughly traced,...,fru_high,None,None,acetylcholine,None,0.576722,None,None,"[AOTU(R), CentralBrain, CentralBrain-unspecifi...","[AOTU(R), CentralBrain, LO(R), LO_R_col_11_13,..."
4,21632,LC10a_R,LC10a,385,1737,2703,1737,606127533,Traced,Roughly traced,...,fru_high,None,None,acetylcholine,None,0.576722,None,None,"[AOTU(R), CentralBrain, CentralBrain-unspecifi...","[AOTU(R), CentralBrain, CentralBrain-unspecifi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,84104,LC10a_R,LC10a,83,274,443,274,143987170,Traced,Roughly traced,...,fru_high,None,None,acetylcholine,None,0.576722,None,None,"[AOTU(R), CentralBrain, CentralBrain-unspecifi...","[AOTU(R), CentralBrain, CentralBrain-unspecifi..."
136,84470,LC10a_R,LC10a,74,248,366,248,143148999,Traced,Roughly traced,...,fru_high,None,None,acetylcholine,None,0.576722,None,None,"[AOTU(R), CentralBrain, CentralBrain-unspecifi...","[AOTU(R), CentralBrain, CentralBrain-unspecifi..."
137,85707,LC10a_R,LC10a,81,222,434,222,140526697,Traced,Roughly traced,...,fru_high,None,None,acetylcholine,None,0.576722,None,None,"[AOTU(R), CentralBrain, CentralBrain-unspecifi...","[AOTU(R), CentralBrain, CentralBrain-unspecifi..."
138,86507,LC10a_R,LC10a,85,380,431,380,139299803,Traced,Roughly traced,...,fru_high,None,None,acetylcholine,None,0.576722,None,None,"[AOTU(R), CentralBrain, CentralBrain-unspecifi...","[AOTU(R), CentralBrain, CentralBrain-unspecifi..."


In [4]:
# Fetch all postsynaptic connections to LC10a_R neurons:
neuron_df, down_conn_df = fetch_adjacencies(LC10a_R_criteria, None) # min_roi_weight = 3, min_total_weight = 5) # you can threshold by listing min_roi_weight or min_total_weight as additional input variable
merged_df = merge_neuron_properties(neuron_df, down_conn_df, ['type','instance'])
merged_df.to_csv(PROJECT_ROOT/'AOTU_results/LC10a_R_AllPostsynapticConnections.csv')
postsynaptic_df = merged_df
postsynaptic_df

,bodyId_pre,bodyId_post,roi,weight,type_pre,instance_pre,type_post,instance_post
0,21065,10005,AOTU(R),128,LC10a,LC10a_R,AOTU019,AOTU019_R
1,21065,10031,AOTU(R),70,LC10a,LC10a_R,AOTU041,AOTU041_R
2,21065,10148,AOTU(R),2,LC10a,LC10a_R,AOTU041,AOTU041_L
3,21065,10212,AOTU(R),4,LC10a,LC10a_R,AOTU042,AOTU042_L
4,21065,10379,AOTU(R),2,LC10a,LC10a_R,DNa10,DNa10_R
...,...,...,...,...,...,...,...,...
21180,100368,141656,LO(R),1,LC10a,LC10a_R,T3,T3_R
21181,100368,151869,AOTU(R),4,LC10a,LC10a_R,AOTU053,AOTU053_R
21182,100368,165007,LO(R),1,LC10a,LC10a_R,Tm5c,Tm5c_R
21183,100368,523317,AOTU(R),3,LC10a,LC10a_R,AOTU041,AOTU041_R


In [5]:
# Fetch all postsynaptic connections to LC10a_R neurons:
neuron_df, up_conn_df = fetch_adjacencies(None, LC10a_R_criteria) # min_roi_weight = 3, min_total_weight = 5) # you can threshold by listing min_roi_weight or min_total_weight as additional input variable
merged_df = merge_neuron_properties(neuron_df, up_conn_df, ['type','instance'])
merged_df.to_csv(PROJECT_ROOT/'AOTU_results/LC10a_R_AllPresynapticConnections.csv')
presynaptic_df = merged_df
presynaptic_df

  0%|          | 0/2 [00:00<?, ?it/s]

,bodyId_pre,bodyId_post,roi,weight,type_pre,instance_pre,type_post,instance_post
0,10005,22397,AOTU(R),1,AOTU019,AOTU019_R,LC10a,LC10a_R
1,10029,23421,LO(R),1,LoVC16,LoVC16_R,LC10a,LC10a_R
2,10029,24035,LO(R),1,LoVC16,LoVC16_R,LC10a,LC10a_R
3,10029,37871,LO(R),1,LoVC16,LoVC16_R,LC10a,LC10a_R
4,10029,42452,LO(R),1,LoVC16,LoVC16_R,LC10a,LC10a_R
...,...,...,...,...,...,...,...,...
37418,862647850,36779,AOTU(R),1,None,None,LC10a,LC10a_R
37419,917042442,22397,LO(R),2,None,None,LC10a,LC10a_R
37420,917042442,23154,LO(R),1,None,None,LC10a,LC10a_R
37421,988188512,26788,LO(R),2,None,None,LC10a,LC10a_R


In [6]:
 # Fetch all postsynaptic connections from right-side LC10a neurons to any neurons in right AOTU, specifically:
neuron_df, AOTU_outputs_df = fetch_adjacencies(LC10a_R_criteria, rois='AOTU(R)') 
merged_df = merge_neuron_properties(neuron_df, AOTU_outputs_df, ['type','instance'])
LC10a_AOTUroi_outputs_df = merged_df
LC10a_AOTUroi_outputs_df.to_csv(PROJECT_ROOT/'AOTU_results/LC10a_R_ALLPostsynapticConnectionsinAOTU_R.csv')
LC10a_AOTUroi_outputs_df

,bodyId_pre,bodyId_post,roi,weight,type_pre,instance_pre,type_post,instance_post
0,21065,10005,AOTU(R),128,LC10a,LC10a_R,AOTU019,AOTU019_R
1,21065,10031,AOTU(R),70,LC10a,LC10a_R,AOTU041,AOTU041_R
2,21065,10148,AOTU(R),2,LC10a,LC10a_R,AOTU041,AOTU041_L
3,21065,10212,AOTU(R),4,LC10a,LC10a_R,AOTU042,AOTU042_L
4,21065,10379,AOTU(R),2,LC10a,LC10a_R,DNa10,DNa10_R
...,...,...,...,...,...,...,...,...
15681,100368,85707,AOTU(R),6,LC10a,LC10a_R,LC10a,LC10a_R
15682,100368,86507,AOTU(R),4,LC10a,LC10a_R,LC10a,LC10a_R
15683,100368,151869,AOTU(R),4,LC10a,LC10a_R,AOTU053,AOTU053_R
15684,100368,523317,AOTU(R),3,LC10a,LC10a_R,AOTU041,AOTU041_R


In [7]:
 # Fetch all postsynaptic connections from right-side LC10a neurons to AOTU neurons (in right or left AOTU):
neuron_df, AOTU_outputs_df = fetch_adjacencies(LC10a_R_criteria, NC(type=('AOTU.*')))#,min_roi_weight=3)
merged_df = merge_neuron_properties(neuron_df, AOTU_outputs_df, ['type','instance'])
LC10a_AOTU_outputs_df = merged_df
LC10a_AOTU_outputs_df.to_csv(PROJECT_ROOT/'AOTU_results/LC10a_R_PostsynapticConnectionstoAOTUs.csv')
LC10a_AOTU_outputs_df

,bodyId_pre,bodyId_post,roi,weight,type_pre,instance_pre,type_post,instance_post
0,21065,10005,AOTU(R),128,LC10a,LC10a_R,AOTU019,AOTU019_R
1,21065,10031,AOTU(R),70,LC10a,LC10a_R,AOTU041,AOTU041_R
2,21065,10148,AOTU(R),2,LC10a,LC10a_R,AOTU041,AOTU041_L
3,21065,10212,AOTU(R),4,LC10a,LC10a_R,AOTU042,AOTU042_L
4,21065,10406,AOTU(R),1,LC10a,LC10a_R,AOTU012,AOTU012_R
...,...,...,...,...,...,...,...,...
5990,100368,37623,AOTU(R),1,LC10a,LC10a_R,AOTU054,AOTU054_R
5991,100368,37829,AOTU(R),9,LC10a,LC10a_R,AOTU059,14368(AOTU)_R
5992,100368,151869,AOTU(R),4,LC10a,LC10a_R,AOTU053,AOTU053_R
5993,100368,523317,AOTU(R),3,LC10a,LC10a_R,AOTU041,AOTU041_R


In [8]:
# Produce Postsynaptic Connection Matrices: will warn that Adriane didn't right all of this code:

# Can change this to any of the connection dataframes above: postsynaptic_df, LC10a_AOTU_outputs, LC10a_AOTUroi_outputs_df
conn_df = LC10a_AOTU_outputs_df 

##inputs
##if instead looking at presynaptic data frame, may want to switch order of these pre and post in teh group_cols list:

#group_cols= ('type_pre', 'type_post') # will compress matrix to only show type-to-type connectivity
group_cols = ('bodyId_pre', 'type_post') # will plot all connectivity of individual right-side LC10a neurons to postsynaptic types
weight_col = 'weight'
sort_by = 'type'
make_square=False

if isinstance(group_cols, str):
    group_cols = (f"{group_cols}_pre", f"{group_cols}_post")

assert len(group_cols) == 2, \
        "Please provide two group_cols (e.g. 'bodyId_pre', 'bodyId_post')"

assert group_cols[0] in conn_df, \
    f"Column missing: {group_cols[0]}"

assert group_cols[1] in conn_df, \
        f"Column missing: {group_cols[1]}"

assert weight_col in conn_df, \
        f"Column missing: {weight_col}"

col_pre, col_post = group_cols
dtype = conn_df[weight_col].dtype

agg_weights_df = conn_df.groupby([col_pre, col_post], sort=False)[weight_col].sum().reset_index()
matrix = agg_weights_df.pivot(index = col_pre, columns = col_post, values = weight_col)
matrix.iloc[:10, :10]


if sort_by:
    if isinstance(sort_by, str):
            sort_by = (f"{sort_by}_pre", f"{sort_by}_post")

    assert len(sort_by) == 2, \
            "Please provide two sort_by column names (e.g. 'type_pre', 'type_post')"

    pre_order = conn_df.sort_values(sort_by[0])[col_pre].unique()
    post_order = conn_df.sort_values(sort_by[1])[col_post].unique()
    matrix = matrix.reindex(index=pre_order, columns=post_order)
else:
    # No sort: Keep the order as close to the input order as possible.
    pre_order = conn_df[col_pre].unique()
    post_order = conn_df[col_post].unique()
    matrix = matrix.reindex(index=pre_order, columns=post_order)

if make_square:
        matrix, _ = matrix.align(matrix.T).fillna(0.0).astype(matrix.dtype)
        matrix = matrix.rename_axis('bodyId_pre', axis=0).rename_axis('bodyId_post', axis=1)
        matrix = matrix.loc[sorted(matrix.index), sorted(matrix.columns)]

matrix.iloc[:10, :10]

matrix.index = matrix.index.astype(str)
matrix.columns = matrix.columns.astype(str)
matrix.hvplot.heatmap(height=800, width=800)

hm = matrix.hvplot.heatmap(height=600, width=1050)
hm * hv.Labels(hm).opts(padding=0)
hm.opts(xrotation=60)
hm.opts(clim=(1, 300))

:HeatMap   [columns,index]   (value)

In [9]:
# Produce Presynaptic Connection Matrices

conn_df = presynaptic_df

#inputs

#group_cols= ('bodyId_pre', 'type_post')
group_cols = ('type_pre', 'type_post')
weight_col = 'weight'
sort_by = 'type'
make_square=False

if isinstance(group_cols, str):
    group_cols = (f"{group_cols}_pre", f"{group_cols}_post")

assert len(group_cols) == 2, \
        "Please provide two group_cols (e.g. 'bodyId_pre', 'bodyId_post')"

assert group_cols[0] in conn_df, \
    f"Column missing: {group_cols[0]}"

assert group_cols[1] in conn_df, \
        f"Column missing: {group_cols[1]}"

assert weight_col in conn_df, \
        f"Column missing: {weight_col}"

col_pre, col_post = group_cols
dtype = conn_df[weight_col].dtype

agg_weights_df = conn_df.groupby([col_pre, col_post], sort=False)[weight_col].sum().reset_index()
matrix = agg_weights_df.pivot(index = col_pre, columns = col_post, values = weight_col)
matrix.iloc[:10, :10]


if sort_by:
    if isinstance(sort_by, str):
            sort_by = (f"{sort_by}_pre", f"{sort_by}_post")

    assert len(sort_by) == 2, \
            "Please provide two sort_by column names (e.g. 'type_pre', 'type_post')"

    pre_order = conn_df.sort_values(sort_by[0])[col_pre].unique()
    post_order = conn_df.sort_values(sort_by[1])[col_post].unique()
    matrix = matrix.reindex(index=pre_order, columns=post_order)
else:
    # No sort: Keep the order as close to the input order as possible.
    pre_order = conn_df[col_pre].unique()
    post_order = conn_df[col_post].unique()
    matrix = matrix.reindex(index=pre_order, columns=post_order)

if make_square:
        matrix, _ = matrix.align(matrix.T).fillna(0.0).astype(matrix.dtype)
        matrix = matrix.rename_axis('bodyId_pre', axis=0).rename_axis('bodyId_post', axis=1)
        matrix = matrix.loc[sorted(matrix.index), sorted(matrix.columns)]

matrix.iloc[:10, :10]

matrix.index = matrix.index.astype(str)
matrix.columns = matrix.columns.astype(str)
matrix.hvplot.heatmap(height=800, width=800)

hm = matrix.hvplot.heatmap(height=600, width=1050)
hm * hv.Labels(hm).opts(padding=0)
hm.opts(clim=(50, 500))

:HeatMap   [columns,index]   (value)

In [10]:
# Plot fraction of LC10a neurons with postsynaptic types in AOTU (Both Sides) -- uses scripts from Reiser Lab Optic Lobe Repo
LC10a_Cxns = CelltypeConnByRoi('LC10a', (['AOTU(R)', 'AOTU(L)']))
LC10a_plots = CelltypeConnPlotter(LC10a_Cxns)
LC10a_plots.plot_consistent_connections_by_type(conn_dir='output')


synapse connection fetched with 0.5 confidence threshold


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/254873 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

cell type connections calculated with 1 threshold


In [ ]:
# Plot fraction of LC10a neurons with postsynaptic types in AOTU (Right side only) -- uses scripts from Reiser Lab Optic Lobe Repo
LC10a_Cxns = CelltypeConnByRoi('LC10a', 'AOTU(R)')
LC10a_plots = CelltypeConnPlotter(LC10a_Cxns)
LC10a_plots.plot_consistent_connections_by_type(conn_dir='output')

In [ ]:
# Plot fraction of LC10a neurons with postsynaptic types in AOTU (left side only) -- uses scripts from Reiser Lab Optic Lobe Repo
LC10a_Cxns = CelltypeConnByRoi('LC10a', 'AOTU(L)')
LC10a_plots = CelltypeConnPlotter(LC10a_Cxns)
LC10a_plots.plot_consistent_connections_by_type(conn_dir='output')

In [ ]:
# Number of synapses from LC10a to each neuron type:  -- uses scripts from Reiser Lab Optic Lobe Repo
LC10a_Cxns = CelltypeConnByRoi('LC10a', (['AOTU(R)', 'AOTU(L)']))
LC10a_plots = CelltypeConnPlotter(LC10a_Cxns)
LC10a_AOTU_out = LC10a_plots.plot_sorted_boxplots_by_conn_type(conn_dir='output', prop_to_plot='syn')

In [ ]:
# Fraction of input received from LC10a to each neuron type:
LC10a_Cxns = CelltypeConnByRoi('LC10a', (['AOTU(R)', 'AOTU(L)']))
LC10a_plots = CelltypeConnPlotter(LC10a_Cxns)
LC10a_AOTU_out = LC10a_plots.plot_sorted_boxplots_by_conn_type(conn_dir='output', prop_to_plot='frac')

In [ ]:
result_dir = PROJECT_ROOT / 'AOTU_results'
result_dir.mkdir(parents=True, exist_ok=True)
param_dir = PROJECT_ROOT / 'params'
result_dir_skel = result_dir / 'cell_gallery'
result_dir_skel.mkdir(parents=True, exist_ok=True)

# Generate list of unique AOTU instances downstream of LC10a_R:
AOTU_bids = pd.unique(LC10a_AOTU_outputs_df.bodyId_post)

fig = plot_cns(bodyid= AOTU_bids, celltype=['None'],show_outline = False, show_skeletons = True, show_shell = True, zoom=3.5)
show_figure(fig,width=800,height=600, showlegend= False)

save_figure(fig
      , name='all_AOTUs'
      , width=1200
      , height=720
    , path=result_dir_skel
    , showlegend=False
    , transparent_bg= True
    ) # adjust size and path


In [ ]:
# Plot Brainwide projection patterns for unique AOTU instances downstream of LC10a_R:

# Fetch list of all primary rois in the cns:
all_rois = fetch_primary_rois()
all_rois.append('NotPrimary')
all_rois.append('OL(L)')
all_rois.append('OL(R)') 
all_rois.append('CentralBrain')
all_rois = sorted(all_rois)

# Generate list of unique AOTU instances downstream of LC10a_R:
criteria = NC(bodyId = pd.unique(LC10a_AOTU_outputs_df.bodyId_post))

neuron_df_all,roi_counts_df = fetch_neurons(criteria)
neuron_df_all = neuron_df_all.sort_values('type')
AOTU_bids = neuron_df_all.bodyId
neurontypes = neuron_df_all.instance

cxn_array = np.zeros(shape=(len(all_rois),len(AOTU_bids)))
total_array = np.zeros(shape = (len(all_rois),1))
for i in AOTU_bids:
    post_vect = np.zeros(shape=(len(all_rois),1)) 
    neuron_df, roi_counts_df = fetch_neurons(NC(bodyId = i))
    for j in roi_counts_df.index: #range(0,len(roi_counts_df.roi)):
        roi_name = roi_counts_df.roi[j]
        if 'LO_R' in roi_name: #find('LO_R_col_18_09') != -1:
                roi_name = 'LO(R)'
        if 'Optic(R)' in roi_name: #find('LO_R_col_18_09') != -1:
                roi_name = 'OL(R)'
        #idx = all_rois.index(roi_counts_df.roi[j])
        idx = all_rois.index(roi_name)
        post_vect[idx] = roi_counts_df.pre[j] # set to .post for upstream, set to .pre for downstream
    total_array = np.concatenate([total_array, post_vect],1)

regions = all_rois
test = pd.DataFrame(data = total_array)#(data = roi_counts_df, columns = ['post'])
import seaborn as sns

#matplotlib inline

roi_df = pd.DataFrame(data = total_array)#(data = roi_counts_df, columns = ['post'])
fig,ax = plt.subplots(figsize = (50,50), dpi=100)
im = ax.imshow(roi_df, vmin = 0, vmax = 40)
ax.set_yticks(np.arange(len(regions)), labels=regions)
ax.set_xticks(np.arange(len(AOTU_bids)), labels = neurontypes)
plt.setp(ax.get_xticklabels(),rotation=45,ha="right", rotation_mode="anchor")
cbar = ax.figure.colorbar(im,ax=ax)
#plt.rcParams['figure.figsize']=[20,20] 
plt.show()

 

In [ ]:
# Use different Package, but for some reason x label length not right anymore...hmph:
roi_df.index = regions
#roi_df.columns = [neurontypes]
roi_df.hvplot.heatmap(height=800, width=800)

hm = roi_df.hvplot.heatmap(height=1200, width=1050)
hm * hv.Labels(hm).opts(padding=0)
hm.opts(clim=(0, 500))